# HowTo -CurveSkeleton- From triangular mesh to curved skeleton

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to transform a triangular mesh in a skeleton
3. how to visualize the skeleton

This notebook **assume** that you have:
- declared the `DB_LOCATION` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
import os

import ipywidgets as widgets
import plotly.graph_objects as go
from plant3dvision.proc3d import skeletonize
from plant3dvision.visu import plotly_mesh, plotly_skeleton, plotly_mesh_data, plotly_skeleton_data
from plantdb.fsdb import FSDB
from plantdb.io import read_triangle_mesh

# Connect to the database & get the initial data

If you did not declare a `DB_LOCATION` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['DB_LOCATION'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['DB_LOCATION'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $DB_LOCATION/<selected_dataset> --config plant-3d-vision/config/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the triangle mesh fileset

The triangular mesh resulting from a _TriangleMesh_ task is to be found in the 'TriangleMesh*' fileset.

In [ ]:
mesh_fs = scan.get_filesets(query={"task_name": "TriangleMesh"})[0]
print(mesh_fs.path().stem)

### Get and load the triangular mesh file

Once you have access to the right fileset, you may access the triangular mesh file and load it as follows:

In [ ]:
mesh_file = mesh_fs.get_files()[0]
tmesh = read_triangle_mesh(mesh_file)

### Visualize the triangular mesh

It is possible to visualize the triangular mesh with the `plotly_mesh` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_mesh(tmesh)

In [ ]:
fig.show()

## Skeletonization

It is now time to process the *triangular mesh* to extract its *skeleton*.

This is done with the `skeletonize` function from `plant3dvision.proc3d` as follows:

In [ ]:
skel = skeletonize(tmesh)

### Visualize the skeleton data

Once the point cloud has been obtained, we can visualize it with the `plotly_skeleton` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_skeleton(skel)

In [ ]:
fig.show()

## Compare the mesh and skeleton

In [ ]:
go_mesh = plotly_mesh_data(tmesh, mesh_kwargs={'opacity': 0.4})
go_skel = plotly_skeleton_data(skel)

In [ ]:
layout_style = dict(height=800, width=800, showlegend=False)
fig = go.Figure(data=[go_mesh] + go_skel, layout=layout_style)
fig.update_scenes(aspectmode='data')
fig.show()

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()